<a href="https://colab.research.google.com/github/nanopiero/fusion/blob/main/notebooks/fcns/training_A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## C11 radar + cmls -> radar + rain gauges 1 min  [xrl_yrg1]
12 out channels are added
At each loop's step:
    - substep 1: segmentation step with the radar mask as target
    - substep 2: regression step with raing gauge data as target 

In [ ]:
# ! git clone https://github.com/nanopiero/fusion.git

In [1]:
import torch
import logging
logger = logging.getLogger("torch")
logger.setLevel(logging.ERROR)  # Show only errors, ignore warnings

import numpy as np
import matplotlib.pyplot as plt
import os
import time
import sys
sys.path.append('/home/mdso/lepetitp/ppc/WEBCAMS/src/raincell/ia/notebooks/learning/simulation')

from fusion.utils.datasets import spatialized_gt, create_cmls_filter, FusionDataset
from fusion.utils.datasets import indices_to_sampled_values, get_point_measurements, point_gt, segment_gt, make_noisy_images
from torch.utils.data import DataLoader
from fusion.utils.fcn import UNet
from fusion.utils.cost_functions import QPELoss_fcn, compute_metrics
from fusion.utils.viz import set_tensor_values2, plot_images, plot_images_10pts_20seg, plot_results_10pts_20seg

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# config de base (change en B.):
num_epochs = 2000
save_every = 10
path = r'/scratch/mdso/lepetitp/ppc/RAINCELL/models/simulation/checkpoint_fcn_exp_C11_xrl_yrg1.pt'
npoints = 10
npairs = 20
nsteps = 60
ndiscs = 5
size_image=64
length_dataset = 6400
device = torch.device('cuda:0')

In [3]:
# Dataset, DataLoader
dataset = FusionDataset(length_dataset=length_dataset,
                        npairs=npairs,
                        nsteps=nsteps,
                        ndiscs=ndiscs, size_image=size_image)


loader = DataLoader(dataset, batch_size=64, num_workers=8)

In [4]:
ch_in = 72
ch_out = nsteps * 3 + 2*12
size = nsteps * 3 

model = UNet(ch_in, ch_out, size, nb_additional_parameters=16).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)

In [5]:
import torch.nn as nn
criterion = QPELoss_fcn()
criterion_segmentation = nn.CrossEntropyLoss()

# Baseline with a FCN
use_fcn = True

best_loss = [float('inf'), float('inf')]  # Initialize best validation loss to a very high value
losses = []
last_epoch = 0

In [ ]:
checkpoint = torch.load(path, \
                            map_location=device)
last_epoch = checkpoint['epoch']
losses = checkpoint['train_losses']
# best_loss = checkpoint['best_loss']
model_weights = checkpoint['model']
optimizer_state_dict = checkpoint['optimizer']
# scheduler_state_dict = checkpoint['scheduler']
model.load_state_dict(model_weights)
optimizer.load_state_dict(optimizer_state_dict)
# scheduler.load_state_dict(scheduler_state_dict)
del checkpoint, model_weights, optimizer_state_dict

In [7]:
last_epoch

0

In [ ]:
import warnings
warnings.filterwarnings("ignore")
model.train()
for epoch in range(last_epoch, num_epochs + 1):
  print('epoch n°', epoch)
  t = time.time()
  running_regression_loss = 0.0
  running_regression_loss_1h = 0.0
  running_segmentation_loss = 0.0
  train_confusion_matrix = np.zeros((2, 2), dtype=int)
  for i, (images, pairs, filters) in enumerate(loader):

    # ground truth (not usable)
    images = images.clone().detach().float().to(device)
    bs = images.shape[0]
      
    # pseudo CMLs
    pairs = pairs.clone().detach().float().to(device)
    filters = filters.clone().float().detach().to(device)

    # for transformers :
    # segment_measurements = segment_gt(images, pairs, filters)
    _, segment_measurements_fcn = segment_gt(images, pairs, filters,
                                             use_fcn=use_fcn)

    # pseudo pluvios
    _, point_measurements_fcn, _ = point_gt(images, npoints=npoints,
                                            use_fcn=use_fcn)

    # pseudo radar
    noisy_images = make_noisy_images(images)

    optimizer.zero_grad()  # Zero the gradients
    # substep 1 : segmentation on radar targets
    inputs = torch.cat([0 * noisy_images, segment_measurements_fcn], dim=1)
    targets = noisy_images
      
    outputs = model(inputs)  # Forward pass
    segmentation_radar_output = torch.cat([outputs[:, -2*12:-12, ...].view(bs,1,12,-1),outputs[:, -12:, ...].view(bs,1,12,-1)], dim=1)
    loss = criterion_segmentation(segmentation_radar_output, targets.long().view(bs,12,-1))
    loss.backward()  # Backward pass
      
    loss_segmentation_radar = loss.item()

      
    # prepare inputs and targets
    inputs = torch.cat([noisy_images, segment_measurements_fcn], dim=1)
    targets = point_measurements_fcn

    outputs = model(inputs)  # Forward pass
    regression_loss, regression_loss_1h, segmentation_loss, loss, batch_cm, _ = criterion(model.p, outputs, targets)
    loss.backward()  # Backward pass

    # substep 2
    # prepare inputs and targets
    inputs = torch.cat([noisy_images, segment_measurements_fcn], dim=1)
    targets = point_measurements_fcn

    outputs = model(inputs)  # Forward pass
    
    regression_loss, regression_loss_1h, segmentation_loss, loss, batch_cm, _ = criterion(model.p, outputs, targets)
    loss.backward()  # Backward pass
    optimizer.step()  # Update the weights

      
    del inputs, targets, outputs, loss, noisy_images, images, pairs, filters
    torch.cuda.empty_cache()
    
    running_regression_loss += regression_loss
    running_regression_loss_1h += regression_loss_1h
    running_segmentation_loss += segmentation_loss
    train_confusion_matrix += batch_cm

  # Calculating average training loss
  train_regression_loss = running_regression_loss / len(loader)
  train_regression_loss_1h = running_regression_loss_1h / len(loader)
  train_segmentation_loss = running_segmentation_loss / len(loader)
  losses.append((epoch, train_regression_loss, train_regression_loss_1h, train_segmentation_loss, train_confusion_matrix, loss_segmentation_radar))
  print(f'Training, Regression Loss: {train_regression_loss:.4f}, Regression Loss 1h: {train_regression_loss_1h:.4f}, Segmentation Loss:{train_segmentation_loss:.4f}' )
  print("Train Confusion Matrix:")
  print(train_confusion_matrix)
  accuracy, csi, sensitivity, specificity, false_alarm_ratio = compute_metrics(train_confusion_matrix)
  print(f'Accuracy: {accuracy:.4f}, CSI: {csi:.4f}, Sensitivity: {sensitivity:.4f}, Specificity: {specificity:.4f}, False Alarm Ratio: {false_alarm_ratio:.4f}')
  print('\n')
  print('epoch duration :', time.time() - t)

  if (epoch % save_every == 0 or \
    epoch == last_epoch):
    print("saving step")
    checkpoint = { 
        'epoch': epoch,
        'model': model.state_dict(),
        'optimizer': optimizer.state_dict(),
        # 'scheduler': scheduler.state_dict(),
        'train_losses': losses,
        }
    torch.save(checkpoint, path)  

epoch n° 0
Training, Regression Loss: 0.1159, Regression Loss 1h: 473.1466, Segmentation Loss:0.2881
Train Confusion Matrix:
[[3081748  111818]
 [ 325824  320610]]
Accuracy: 0.8860, CSI: 0.4228, Sensitivity: 0.4960, Specificity: 0.9650, False Alarm Ratio: 0.2586


epoch duration : 192.10221934318542
saving step
epoch n° 1
Training, Regression Loss: 0.0768, Regression Loss 1h: 253.8390, Segmentation Loss:0.2114
Train Confusion Matrix:
[[3098768  101145]
 [ 196603  443484]]
Accuracy: 0.9225, CSI: 0.5983, Sensitivity: 0.6928, Specificity: 0.9684, False Alarm Ratio: 0.1857


epoch duration : 188.20910620689392
epoch n° 2
Training, Regression Loss: 0.0650, Regression Loss 1h: 226.7098, Segmentation Loss:0.1957
Train Confusion Matrix:
[[3096573   97328]
 [ 179088  467011]]
Accuracy: 0.9280, CSI: 0.6282, Sensitivity: 0.7228, Specificity: 0.9695, False Alarm Ratio: 0.1725


epoch duration : 185.92811179161072
epoch n° 3
Training, Regression Loss: 0.0608, Regression Loss 1h: 215.5366, Segmentat